# Model 3: Optimizer

This notebook contains the construction and training of the model iterations and various experiments. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the *model_optimization_and_evaluation.ipynb* notebook found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
elif training_mode == 'local':
    data_path = data_path_local
    model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
batch_size = 32
n_classes = 20
n_epochs = 15
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1 - SGD

In [6]:
# defining the model architecture
m3_t1 = models.Sequential()

# convolution/max pool stacks
m3_t1.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m3_t1.add(layers.MaxPooling2D((2,2)))

m3_t1.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m3_t1.add(layers.MaxPooling2D((2,2)))

m3_t1.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m3_t1.add(layers.MaxPooling2D((2,2)))

m3_t1.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m3_t1.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m3_t1.add(layers.Flatten())
m3_t1.add(layers.Dense(512, activation='relu'))
m3_t1.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m3_t1.summary()
model_names.append('m3_t1')
model_list.append(m3_t1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 256)      

### Trial 2 - RMSProp

In [7]:
# defining the model architecture
m3_t2 = models.Sequential()

# convolution/max pool stacks
m3_t2.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m3_t2.add(layers.MaxPooling2D((2,2)))

m3_t2.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m3_t2.add(layers.MaxPooling2D((2,2)))

m3_t2.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m3_t2.add(layers.MaxPooling2D((2,2)))

m3_t2.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m3_t2.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m3_t2.add(layers.Flatten())
m3_t2.add(layers.Dense(512, activation='relu'))
m3_t2.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m3_t2.summary()
model_names.append('m3_t2')
model_list.append(m3_t2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 256)      

### Trial 3 - Adam

In [8]:
# defining the model architecture
m3_t3 = models.Sequential()

# convolution/max pool stacks
m3_t3.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m3_t3.add(layers.MaxPooling2D((2,2)))

m3_t3.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m3_t3.add(layers.MaxPooling2D((2,2)))

m3_t3.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m3_t3.add(layers.MaxPooling2D((2,2)))

m3_t3.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m3_t3.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m3_t3.add(layers.Flatten())
m3_t3.add(layers.Dense(512, activation='relu'))
m3_t3.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m3_t3.summary()
model_names.append('m3_t3')
model_list.append(m3_t3)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 25, 25, 256)      

### Trial 4 - Adadelta

In [9]:
# defining the model architecture
m3_t4 = models.Sequential()

# convolution/max pool stacks
m3_t4.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m3_t4.add(layers.MaxPooling2D((2,2)))

m3_t4.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m3_t4.add(layers.MaxPooling2D((2,2)))

m3_t4.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m3_t4.add(layers.MaxPooling2D((2,2)))

m3_t4.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m3_t4.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m3_t4.add(layers.Flatten())
m3_t4.add(layers.Dense(512, activation='relu'))
m3_t4.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m3_t4.summary()
model_names.append('m3_t4')
model_list.append(m3_t4)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 25, 25, 256)      

### Trial 5 - Adagrad

In [10]:
# defining the model architecture
m3_t5 = models.Sequential()

# convolution/max pool stacks
m3_t5.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m3_t5.add(layers.MaxPooling2D((2,2)))

m3_t5.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m3_t5.add(layers.MaxPooling2D((2,2)))

m3_t5.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m3_t5.add(layers.MaxPooling2D((2,2)))

m3_t5.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m3_t5.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m3_t5.add(layers.Flatten())
m3_t5.add(layers.Dense(512, activation='relu'))
m3_t5.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m3_t5.summary()
model_names.append('m3_t5')
model_list.append(m3_t5)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 25, 25, 256)      

### Trial 6 - Adamax

In [11]:
# defining the model architecture
m3_t6 = models.Sequential()

# convolution/max pool stacks
m3_t6.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m3_t6.add(layers.MaxPooling2D((2,2)))

m3_t6.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m3_t6.add(layers.MaxPooling2D((2,2)))

m3_t6.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m3_t6.add(layers.MaxPooling2D((2,2)))

m3_t6.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m3_t6.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m3_t6.add(layers.Flatten())
m3_t6.add(layers.Dense(512, activation='relu'))
m3_t6.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m3_t6.summary()
model_names.append('m3_t6')
model_list.append(m3_t6)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 25, 25, 256)      

### Trial 7 - Nadam

In [12]:
# defining the model architecture
m3_t7 = models.Sequential()

# convolution/max pool stacks
m3_t7.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
m3_t7.add(layers.MaxPooling2D((2,2)))

m3_t7.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
m3_t7.add(layers.MaxPooling2D((2,2)))

m3_t7.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
m3_t7.add(layers.MaxPooling2D((2,2)))

m3_t7.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
m3_t7.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m3_t7.add(layers.Flatten())
m3_t7.add(layers.Dense(512, activation='relu'))
m3_t7.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m3_t7.summary()
model_names.append('m3_t7')
model_list.append(m3_t7)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 25, 25, 256)      

## Step 4: Training

In [13]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
%%time
# data standardization and augmentation
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

CPU times: user 268 ms, sys: 1.06 s, total: 1.33 s
Wall time: 1.33 s


In [15]:
# compiling loss functions
m3_t1.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['acc'])
m3_t2.compile(loss='categorical_crossentropy', optimizer='RMSProp', metrics=['acc'])
m3_t3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
m3_t4.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc'])
m3_t5.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['acc'])
m3_t6.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])
m3_t7.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['acc'])

In [16]:
%%time
# setting up model saving checkpoints
m3_t1_cp = ModelCheckpoint(filepath=f'{model_path}m3_t1.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m3_t1_history = m3_t1.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m3_t1_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m3_t1_history)

Epoch 1/15
253/253 [==============================] - 134s 529ms/step - loss: 2.9718 - acc: 0.0867 - val_loss: 2.9650 - val_acc: 0.0825

Epoch 00001: val_acc improved from -inf to 0.08250, saving model to /floyd/home/m3_t1.h5
Epoch 2/15
253/253 [==============================] - 72s 286ms/step - loss: 2.9370 - acc: 0.0968 - val_loss: 2.8417 - val_acc: 0.1260

Epoch 00002: val_acc improved from 0.08250 to 0.12596, saving model to /floyd/home/m3_t1.h5
Epoch 3/15
253/253 [==============================] - 73s 290ms/step - loss: 2.7659 - acc: 0.1300 - val_loss: 2.6282 - val_acc: 0.1838

Epoch 00003: val_acc improved from 0.12596 to 0.18380, saving model to /floyd/home/m3_t1.h5
Epoch 4/15
253/253 [==============================] - 72s 286ms/step - loss: 2.6400 - acc: 0.1498 - val_loss: 2.4881 - val_acc: 0.1915

Epoch 00004: val_acc improved from 0.18380 to 0.19152, saving model to /floyd/home/m3_t1.h5
Epoch 5/15
253/253 [==============================] - 74s 291ms/step - loss: 2.5727 - acc:

In [17]:
%%time
# setting up model saving checkpoints
m3_t2_cp = ModelCheckpoint(filepath=f'{model_path}m3_t2.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m3_t2_history = m3_t2.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m3_t2_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m3_t2_history)

Epoch 1/15
253/253 [==============================] - 74s 291ms/step - loss: 2.9375 - acc: 0.1124 - val_loss: 2.6489 - val_acc: 0.1900

Epoch 00001: val_acc improved from -inf to 0.19000, saving model to /floyd/home/m3_t2.h5
Epoch 2/15
253/253 [==============================] - 73s 290ms/step - loss: 2.4864 - acc: 0.2036 - val_loss: 2.1616 - val_acc: 0.2712

Epoch 00002: val_acc improved from 0.19000 to 0.27121, saving model to /floyd/home/m3_t2.h5
Epoch 3/15
253/253 [==============================] - 74s 294ms/step - loss: 2.2883 - acc: 0.2650 - val_loss: 1.9439 - val_acc: 0.3175

Epoch 00003: val_acc improved from 0.27121 to 0.31748, saving model to /floyd/home/m3_t2.h5
Epoch 4/15
253/253 [==============================] - 74s 293ms/step - loss: 2.1794 - acc: 0.3062 - val_loss: 1.9158 - val_acc: 0.2982

Epoch 00004: val_acc did not improve from 0.31748
Epoch 5/15
253/253 [==============================] - 74s 292ms/step - loss: 2.1159 - acc: 0.3284 - val_loss: 1.6203 - val_acc: 0.422

In [18]:
%%time
# setting up model saving checkpoints
m3_t3_cp = ModelCheckpoint(filepath=f'{model_path}m3_t3.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m3_t3_history = m3_t3.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m3_t3_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m3_t3_history)

Epoch 1/15
253/253 [==============================] - 73s 290ms/step - loss: 2.8488 - acc: 0.1112 - val_loss: 2.2923 - val_acc: 0.1700

Epoch 00001: val_acc improved from -inf to 0.17000, saving model to /floyd/home/m3_t3.h5
Epoch 2/15
253/253 [==============================] - 73s 288ms/step - loss: 2.5396 - acc: 0.1755 - val_loss: 2.3455 - val_acc: 0.2481

Epoch 00002: val_acc improved from 0.17000 to 0.24807, saving model to /floyd/home/m3_t3.h5
Epoch 3/15
253/253 [==============================] - 73s 287ms/step - loss: 2.3964 - acc: 0.2169 - val_loss: 1.9619 - val_acc: 0.2905

Epoch 00003: val_acc improved from 0.24807 to 0.29049, saving model to /floyd/home/m3_t3.h5
Epoch 4/15
253/253 [==============================] - 72s 286ms/step - loss: 2.2664 - acc: 0.2507 - val_loss: 2.0215 - val_acc: 0.3573

Epoch 00004: val_acc improved from 0.29049 to 0.35733, saving model to /floyd/home/m3_t3.h5
Epoch 5/15
253/253 [==============================] - 72s 284ms/step - loss: 2.1698 - acc: 

In [19]:
%%time
# setting up model saving checkpoints
m3_t4_cp = ModelCheckpoint(filepath=f'{model_path}m3_t4.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m3_t4_history = m3_t4.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m3_t4_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m3_t4_history)

Epoch 1/15
253/253 [==============================] - 73s 287ms/step - loss: 2.7992 - acc: 0.1165 - val_loss: 2.4084 - val_acc: 0.1612

Epoch 00001: val_acc improved from -inf to 0.16125, saving model to /floyd/home/m3_t4.h5
Epoch 2/15
253/253 [==============================] - 72s 285ms/step - loss: 2.5447 - acc: 0.1673 - val_loss: 2.2788 - val_acc: 0.2339

Epoch 00002: val_acc improved from 0.16125 to 0.23393, saving model to /floyd/home/m3_t4.h5
Epoch 3/15
253/253 [==============================] - 73s 287ms/step - loss: 2.4040 - acc: 0.2211 - val_loss: 2.4314 - val_acc: 0.2841

Epoch 00003: val_acc improved from 0.23393 to 0.28406, saving model to /floyd/home/m3_t4.h5
Epoch 4/15
253/253 [==============================] - 72s 286ms/step - loss: 2.2662 - acc: 0.2630 - val_loss: 2.3096 - val_acc: 0.3239

Epoch 00004: val_acc improved from 0.28406 to 0.32391, saving model to /floyd/home/m3_t4.h5
Epoch 5/15
253/253 [==============================] - 72s 285ms/step - loss: 2.1711 - acc: 

In [20]:
%%time
# setting up model saving checkpoints
m3_t5_cp = ModelCheckpoint(filepath=f'{model_path}m3_t5.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m3_t5_history = m3_t5.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m3_t5_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m3_t5_history)

Epoch 1/15
253/253 [==============================] - 72s 286ms/step - loss: 6.4759 - acc: 0.1093 - val_loss: 2.4493 - val_acc: 0.1800

Epoch 00001: val_acc improved from -inf to 0.18000, saving model to /floyd/home/m3_t5.h5
Epoch 2/15
253/253 [==============================] - 72s 284ms/step - loss: 2.5433 - acc: 0.1833 - val_loss: 2.1734 - val_acc: 0.2648

Epoch 00002: val_acc improved from 0.18000 to 0.26478, saving model to /floyd/home/m3_t5.h5
Epoch 3/15
253/253 [==============================] - 72s 284ms/step - loss: 2.3922 - acc: 0.2275 - val_loss: 2.0515 - val_acc: 0.2931

Epoch 00003: val_acc improved from 0.26478 to 0.29306, saving model to /floyd/home/m3_t5.h5
Epoch 4/15
253/253 [==============================] - 72s 283ms/step - loss: 2.2814 - acc: 0.2629 - val_loss: 2.0831 - val_acc: 0.3406

Epoch 00004: val_acc improved from 0.29306 to 0.34062, saving model to /floyd/home/m3_t5.h5
Epoch 5/15
253/253 [==============================] - 72s 284ms/step - loss: 2.1815 - acc: 

In [21]:
%%time
# setting up model saving checkpoints
m3_t6_cp = ModelCheckpoint(filepath=f'{model_path}m3_t6.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m3_t6_history = m3_t6.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m3_t6_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m3_t6_history)

Epoch 1/15
253/253 [==============================] - 74s 291ms/step - loss: 2.7881 - acc: 0.1211 - val_loss: 2.3680 - val_acc: 0.1800

Epoch 00001: val_acc improved from -inf to 0.18000, saving model to /floyd/home/m3_t6.h5
Epoch 2/15
253/253 [==============================] - 74s 292ms/step - loss: 2.5053 - acc: 0.1913 - val_loss: 2.1748 - val_acc: 0.3046

Epoch 00002: val_acc improved from 0.18000 to 0.30463, saving model to /floyd/home/m3_t6.h5
Epoch 3/15
253/253 [==============================] - 73s 290ms/step - loss: 2.2947 - acc: 0.2505 - val_loss: 2.6751 - val_acc: 0.3406

Epoch 00003: val_acc improved from 0.30463 to 0.34062, saving model to /floyd/home/m3_t6.h5
Epoch 4/15
253/253 [==============================] - 74s 292ms/step - loss: 2.1700 - acc: 0.3003 - val_loss: 2.1819 - val_acc: 0.3483

Epoch 00004: val_acc improved from 0.34062 to 0.34833, saving model to /floyd/home/m3_t6.h5
Epoch 5/15
253/253 [==============================] - 74s 291ms/step - loss: 2.0713 - acc: 

In [22]:
%%time
# setting up model saving checkpoints
m3_t7_cp = ModelCheckpoint(filepath=f'{model_path}m3_t7.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m3_t7_history = m3_t7.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m3_t7_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m3_t7_history)

Epoch 1/15
253/253 [==============================] - 74s 292ms/step - loss: 3.0099 - acc: 0.0931 - val_loss: 2.8263 - val_acc: 0.1450

Epoch 00001: val_acc improved from -inf to 0.14500, saving model to /floyd/home/m3_t7.h5
Epoch 2/15
253/253 [==============================] - 73s 288ms/step - loss: 2.8788 - acc: 0.1103 - val_loss: 2.7854 - val_acc: 0.1504

Epoch 00002: val_acc improved from 0.14500 to 0.15039, saving model to /floyd/home/m3_t7.h5
Epoch 3/15
253/253 [==============================] - 73s 288ms/step - loss: 2.8166 - acc: 0.1337 - val_loss: 2.6140 - val_acc: 0.1787

Epoch 00003: val_acc improved from 0.15039 to 0.17866, saving model to /floyd/home/m3_t7.h5
Epoch 4/15
253/253 [==============================] - 72s 286ms/step - loss: 2.5283 - acc: 0.1768 - val_loss: 2.3470 - val_acc: 0.2339

Epoch 00004: val_acc improved from 0.17866 to 0.23393, saving model to /floyd/home/m3_t7.h5
Epoch 5/15
253/253 [==============================] - 73s 289ms/step - loss: 2.4352 - acc: 

In [23]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [24]:
# evaluating models on validation set
for key, value in models_dict.items():
    model = models.load_model(f'{model_path}{key}.h5')
    (val_loss, val_accuracy) = model.evaluate(val_generator,verbose=1)
    print(f'{key} Val Accuracy: {round((val_accuracy*100),2)}%')
    print(f'{key} Val Loss: {round(val_loss,4)}')
    print('---')

26/26 [==============================] - 1s 44ms/step
m3_t1 Val Accuracy: 39.75%
m3_t1 Val Loss: 2.086
---
26/26 [==============================] - 1s 43ms/step
m3_t2 Val Accuracy: 50.49%
m3_t2 Val Loss: 2.0096
---
26/26 [==============================] - 1s 43ms/step
m3_t3 Val Accuracy: 50.37%
m3_t3 Val Loss: 1.7346
---
26/26 [==============================] - 1s 44ms/step
m3_t4 Val Accuracy: 51.6%
m3_t4 Val Loss: 1.7007
---
26/26 [==============================] - 1s 46ms/step
m3_t5 Val Accuracy: 51.6%
m3_t5 Val Loss: 2.0198
---
26/26 [==============================] - 1s 43ms/step
m3_t6 Val Accuracy: 53.83%
m3_t6 Val Loss: 1.6315
---
26/26 [==============================] - 1s 43ms/step
m3_t7 Val Accuracy: 42.47%
m3_t7 Val Loss: 2.0155
---


In [25]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

m3_t1_history saved in /floyd/home/
m3_t2_history saved in /floyd/home/
m3_t3_history saved in /floyd/home/
m3_t4_history saved in /floyd/home/
m3_t5_history saved in /floyd/home/
m3_t6_history saved in /floyd/home/
m3_t7_history saved in /floyd/home/
